In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Explore swaps convexity



In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.market.curves import *
from financepy.utils.day_count import DayCount, DayCountTypes

import financepy.products.rates.ibor_curve_risk_engine as re
from financepy.products.rates.ibor_benchmarks_report import dataframe_to_benchmarks

### Print dataframes a certain way

In [9]:
pd.set_option('display.max_rows', None,
                    'display.max_columns', None,
                    'display.float_format', lambda x:f'{x:.4f}'
                    )

### Function to plot a curve

In [10]:

def plotCurve(curve, start_date, tmax, instr_mat_dates_or_tenor = None, title = ''):
    years = np.linspace(1/365, tmax, int(365*tmax)//30)
    dates = start_date.add_years(years)
    zero_rates = curve.zero_rate(dates)
    on_fwd_rates = curve.fwd(dates)

    ys_monthly = np.arange(1,tmax*12+1)/12
    monthly_dates = start_date.add_years(ys_monthly)

    if instr_mat_dates_or_tenor is not None:
        # Will plot term forward rates
        # instr_mat_dates_or_tenor could be a list of curve-building
        # isntrument maturities in which case term fwd rates go from the last instrument maturiy date that is less
        # than the plotting date to the plotting date. Or it could be a tenor so that fwd rates cover that tenor 
        # i.e. the term fwd rate for d covers [max(d-tenor,start_date), d]
        # tenor should be a positive tenor

        if isinstance(instr_mat_dates_or_tenor, str):
            neg_tenor = '-' + instr_mat_dates_or_tenor
            start_fwd_dates = [d.add_tenor(neg_tenor) for d in dates]
            start_fwd_dates = [d if d > start_date else start_date for d in start_fwd_dates]
            fwd_rate_label = f'term fwd rates for [d-{instr_mat_dates_or_tenor},d]'
        else:
            instr_mat_dates_or_tenor = [start_date] + instr_mat_dates_or_tenor
            start_fwd_dates = [ max([md  for md in instr_mat_dates_or_tenor if md < d]) for d in dates]
            fwd_rate_label = 'term fwd rates from prev instr mtrty'

        term_fwd_rates = curve.fwd_rate(start_fwd_dates, dates)

    plt.figure(figsize=(8,6))
    plt.plot(years, zero_rates*100, '-', label="zero rates")
    plt.plot(years, on_fwd_rates*100, '-', label = "ON fwd rates")

    if instr_mat_dates_or_tenor is not None:
        plt.plot(years, term_fwd_rates*100, '.', label = fwd_rate_label)

    plt.xlabel("Times in years")
    plt.ylabel("Rates (%) - See Legend")
    plt.title(title)
    plt.legend()

### Load csv

In [11]:
filename = './data/bms_GBP_SONIA_20220901.csv'
dfbm = pd.read_csv(filename, index_col = 0)
dfbm['base_date']=pd.to_datetime(dfbm['base_date'], errors = 'ignore', format = '%d/%m/%Y')
dfbm['start_date']=pd.to_datetime(dfbm['start_date'], errors = 'ignore', format = '%d/%m/%Y')
dfbm['maturity_date']=pd.to_datetime(dfbm['maturity_date'], errors = 'ignore', format = '%d/%m/%Y')

valuation_date = date.from_datetime(dfbm.loc[0,'base_date'])
cal = CalendarTypes.UNITED_KINGDOM
bms = dataframe_to_benchmarks(dfbm, asof_date=valuation_date,calendar_type=cal)
depos = bms['IborDeposit']
fras = bms['IborFRA']
swaps = bms['IborSwap']

fras.sort( key = lambda fra:fra.maturity_dt)

remove_mpc_fras = True
if remove_mpc_fras:
    fras_to_use = [f for f in fras if date.datediff(f.start_dt,f.maturity_dt) > 60]
    fras = fras_to_use

last_year = 30
swaps_to_use = [s for s in swaps if DayCount(DayCountTypes.SIMPLE).year_frac(s.effective_dt,s.maturity_dt)[0] <=  last_year]
swaps = swaps_to_use

curve = IborSingleCurve(valuation_date, depos, fras, swaps, InterpTypes.PCHIP_ZERO_RATES)
plotCurve(curve, valuation_date, 20, instr_mat_dates_or_tenor='1Y')

FileNotFoundError: [Errno 2] No such file or directory: './data/bms_GBP_SONIA_20220901.csv'

### Some common params

In [9]:
spot_days = 2
settlement_date = valuation_date.add_weekdays(spot_days)
fixedDCCType = DayCountTypes.ACT_365F
fixedFreqType = FrequencyTypes.ANNUAL


### A function to create a RECEIVE position of specified maturity hedged by a short-maturity swap

In [16]:
def hedged_position(maturity : str, crv = curve):
    vdate = crv.value_dt
    nt = 10_000
    s = IborSwap(settlement_date, maturity, SwapTypes.RECEIVE, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = nt)
    s.set_fixed_rate_to_atm(valuation_date, discount_curve = crv)    
    h1 = IborSwap(settlement_date, '1Y', SwapTypes.RECEIVE, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = nt)
    pv01_s= s.pv01(vdate, discount_curve = crv)
    pv01_h = h1.pv01(vdate, discount_curve = crv)
    h =  IborSwap(settlement_date, '1Y', SwapTypes.PAY, 1*g_percent, fixedFreqType, fixedDCCType, cal_type=cal, notional = nt * pv01_s/pv01_h)
    h.set_fixed_rate_to_atm(valuation_date, discount_curve = crv)    
    return [s,h]

### Long-convexity hedged-swap positions at different maturities

In [ ]:
curve_shifts = np.linspace(-200,200,19,endpoint=True)
f = plt.figure()
for n in np.arange(1,51,10):
    pos = hedged_position(f'{n}Y')
    bv, rr = re.parallel_shift_ladder_report(curve, curve_shifts*g_basis_point,  pos, )
    plt.plot(curve_shifts, rr[re.PV_PREFIX + 'total'], label = f'mty={n}Y')
plt.legend(loc = 'best')
plt.xlabel('curve shift, bp')
plt.title('PnL for a hedged receiver of given maturity')
plt.show()
    